# Testing Several Different Prompting Methodologies (via SOLAR 10.7B)


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Upstage/SOLAR-10.7B-Instruct-v1.0")
model = AutoModelForCausalLM.from_pretrained(
    "Upstage/SOLAR-10.7B-Instruct-v1.0",
    device_map="auto",
    torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

## Zero-Shot + Instruction Prompting
- Simply feed the task text to the model and ask for results.

In [ ]:
zero_shot = [
    {
        "role": "user",
        "content": "Choose 3 from the following descriptions that match the name 'Snowflake': Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming",
    }
]

prompt = tokenizer.apply_chat_template(
    zero_shot, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, use_cache=True, max_length=4096)
output_text = tokenizer.decode(outputs[0])
print(output_text)

<s> ### User:
Choose 3 from the following descriptions that match the name 'Snowflake': Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming

### Assistant:
1. Playful
2. Gentle
3. Majestic</s>


## Few-Shot + Instruction Prompting
- Feed a set of high-quality demonstrations, each consisting of both input and desired output, on the target task.
- Likely to feed similar names : use k-NN in the embedding space ([Liu et al., 2021](https://arxiv.org/abs/2101.06804))
- Constructing good example set might be another task.

In [ ]:
few_shot = [
    {
        "role": "user",
        "content": "Based on the examples below, choose 3 from the following descriptions that match the name 'Snowflake': Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming\n\nExample 1:\nName: Snowwhite\nChosen Descriptions: Playful, Gentle, Majestic\n\nExample 2:\nName: Snowy\nChosen Descriptions: Curious, Loyal, Witty",
    }
]

prompt = tokenizer.apply_chat_template(
    few_shot, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, use_cache=True, max_length=4096) 
output_text = tokenizer.decode(outputs[0])
print(output_text)

## Few-Shot + Instruction Prompting (alternative)
__*Using different tasks as example sets.*__
- Repeating the same task as an example is not necessary.

In [ ]:
few_shot_alt = [
    {
        "role": "user",
        "content": "Based on the examples below, choose 3 from the following descriptions that match the name 'Snowflake': Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming\n\nExample 1:\nName: Snow\nChosen Descriptions: White, Bright, Romantic\n\nExample 2:\nName: Winter\nChosen Descriptions: Frosty, Cold, Serene\n\nExample 3:\nName: Christmas\nChosen Descriptions: Romantic, Cold, Serene",
    }
]

prompt = tokenizer.apply_chat_template(
    few_shot_alt, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, use_cache=True, max_length=4096) 
output_text = tokenizer.decode(outputs[0])
print(output_text)

## Self-Consistency Sampling : [Wang et al., 2022a](https://arxiv.org/pdf/2203.11171.pdf)
- Sample diverse answers through different reasoning paths, then apply majority vote.
- $temperature > 0$


In [ ]:
sc_sampling = [
    {
        "role": "user",
        "content": "Choose 3 from the following descriptions that match the name 'Snowflake': Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming. Now, using self-consistency sampling, describe a fictional or idealized version of Snowflake, incorporating those chosen characteristics into a cohesive and vivid portrayal.",
    }
]

sc_sampling_alt = [
    {
        "role": "user",
        "content": "By the method below, choose 3 from the following descriptions that match the name 'Snowflake': Playful, Gentle, Majestic, Curious, Loyal, Witty, Mysterious, Adventurous, Elegant, Charming\n\nFirst, pick the very best description that match name about 20 times, use different reasoning pathway each.\n\nThen show me the top three most frequently chosen descriptions. Description is not needed.",
    }
]

prompt = tokenizer.apply_chat_template(
    sc_sampling, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, use_cache=True, max_length=4096)
output_text = tokenizer.decode(outputs[0])
print(output_text)

__Citation :__
[Lilian Weng, 2023](https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/)